In [3]:

import numpy as np
import torch
from cebmf_torch import *
 
rng = np.random.default_rng(42)
n,p=50,40
u = rng.random(n); v=rng.random(p)
X = np.outer(u,v) + rng.normal(0,0.1,(n,p))
X[1,1]=np.nan; X[10,5]=np.nan; X[30:33,12:14]=np.nan
Y = torch.tensor(X, dtype=torch.float32)
m = cEBMF(Y, K=5, prior_L="norm", prior_F="norm")
m.initialize("svd")

    # Ensure no nans in learned factors and tau positive


In [ ]:
  mask_f = self.mask if self.mask.dtype.is_floating_point else self.mask.to(self.L.dtype)

        Lk  = self.L[:, k]    # (N,)
        Fk  = self.F[:, k]    # (P,)
        Lk2 = self.L2[:, k]   # (N,)
        Fk2 = self.F2[:, k]   # (P,)

        # ---- 1) Add back k's contribution: R <- R + Lk Fk^T  (no outer allocation) ----
        # This forms the partial residual that keeps all components except k removed.
        self.R.addr_(Lk, Fk, alpha=1.0)
        self.R.mul_(mask_f)
        with torch.no_grad():  # ---------- Update L[:, k] ----------
            if tau_map is None:
                denom_l = mask_f @ Fk2                             # (N,)
                # num_l[i]   = sum_j R[i,j] * Fk[j]
                num_l   = self.R @ Fk                              # (N,)
                se_l    = torch.sqrt(1.0 / (self.tau * denom_l.clamp_min(eps)))
            else:
                # denom_l[i] = sum_j tau_map[i,j]*mask[i,j]*Fk2[j]
                denom_l = (tau_map * mask_f) @ Fk2                 # (N,)
            # num_l[i]   = sum_j tau_map[i,j]*R[i,j]*Fk[j]   (no (tau_map*R) buffer)
                num_l   = torch.einsum('ij,ij,j->i', self.R, tau_map, Fk)  # (N,)
                se_l    = torch.sqrt(1.0 / denom_l.clamp_min(eps))

            lhat = num_l / denom_l
        # print(denom_l)

        X_model = self.update_cov_L(k)
        with torch.enable_grad():
            resL = self.prior_L_fn.fit(
                X=X_model,
                betahat=lhat,
                sebetahat=se_l,
                model_param=self.model_state_L[k],
            )
        with torch.no_grad():
            self.model_state_L[k] = resL.model_param
            self.L[:, k] = resL.post_mean
            self.L2[:, k] = resL.post_mean2
            nm_ll_L = normal_means_loglik(x=lhat, s=se_l, Et=resL.post_mean, Et2=resL.post_mean2)
            self.kl_l[k] = torch.as_tensor((-resL.loss) - nm_ll_L, device=self.device)
            self.pi0_L[k] = resL.pi0_null if hasattr(resL, "pi0_null") else None

            # ---------- Update F[:, k] ----------
            # ---- 2) Subtract UPDATED L_k: R <- R - Lk_new Fk^T ----
            Lk = self.L[:, k]  # updated
            lk2 = self.L2[:, k]
            self.R.addr_(Lk, Fk, alpha=-1.0)
            self.R.mul_(mask_f)

    # ---- 3) Add back (updated L_k) again to prepare F update: R <- R + Lk_new Fk^T ----
            self.R.addr_(Lk, Fk, alpha=1.0)
            self.R.mul_(mask_f)
          

        with torch.no_grad():
            if tau_map is None:
                # denom_f[j] = sum_i mask[i,j] * Lk2[i]
                denom_f = mask_f.T @ self.L2[:, k]                 # (P,)
                # num_f[j]   = sum_i R[i,j] * Lk[i]
                num_f   = self.R.T @ Lk                            # (P,)
                se_f    = torch.sqrt(1.0 / (self.tau * denom_f.clamp_min(eps)))
            else:
                # denom_f[j] = sum_i tau_map[i,j]*mask[i,j]*Lk2[i]
                denom_f = (tau_map * mask_f).transpose(0, 1) @ self.L2[:, k]  # (P,)
            # num_f[j]   = sum_i tau_map[i,j]*R[i,j]*Lk[i]
                num_f   = torch.einsum('ij,ij,i->j', self.R, tau_map, Lk)     # (P,)
                se_f    = torch.sqrt(1.0 / denom_f.clamp_min(eps))

            fhat = num_f / denom_f.clamp_min(eps)

        X_model = self.update_cov_F(k)
        with torch.enable_grad():
            resF = self.prior_F_fn.fit(
                X=X_model,
                betahat=fhat,
                sebetahat=se_f,
                model_param=self.model_state_F[k],
            )
        with torch.no_grad():
            self.model_state_F[k] = resF.model_param
            self.F[:, k] = resF.post_mean
            self.F2[:, k] = resF.post_mean2
            # store as scalar on device; PriorResult.loss already = -log_lik
            nm_ll_F = normal_means_loglik(x=fhat, s=se_f, Et=resF.post_mean, Et2=resF.post_mean2)
            self.kl_f[k] = torch.as_tensor((-resF.loss) - nm_ll_F, device=self.device)
            self.pi0_F[k] = resF.pi0_null if hasattr(resF, "pi0_null") else None

            Fk = self.F[:, k]  # updated
            self.R.addr_(Lk, Fk, alpha=-1.0)
            self.R.mul_(mask_f)


In [4]:
torch.isfinite(m.L).all() 



tensor(True)

In [ ]:
torch.isfinite(m.F).all()

In [ ]:
float(m.tau) > 0